In [1]:
!python --version

Python 3.5.2 :: Anaconda 4.2.0 (x86_64)


In [2]:
import pandas as pd

### Import and Explore Data

In [3]:
import sqlite3

Source: https://www.kaggle.com/worldbank/world-development-indicators

In [4]:
connection = sqlite3.connect("/Users/Alexandre/Workstation/databases/world_development_indicators.db")

In [5]:
cursor = connection.cursor()

What tables do we have?

In [6]:
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)

In [7]:
df

,name
0,Country
1,CountryNotes
2,Footnotes
3,Indicators
4,Series
5,SeriesNotes


In [8]:
pd.read_sql_query("SELECT * FROM Country LIMIT 1", connection).set_index('CountryCode').sort_index().drop('index', 1).transpose()

CountryCode,AFG
ShortName,Afghanistan
TableName,Afghanistan
LongName,Islamic State of Afghanistan
Alpha2Code,AF
CurrencyUnit,Afghan afghani
SpecialNotes,Fiscal year end: March 20; reporting period fo...
Region,South Asia
IncomeGroup,Low income
Wb2Code,AF
NationalAccountsBaseYear,2002/03


In [9]:
regions_df = pd.read_sql_query("SELECT TableName, Region FROM Country", connection)
regions_df.head()

,TableName,Region
0,Afghanistan,South Asia
1,Albania,Europe & Central Asia
2,Algeria,Middle East & North Africa
3,American Samoa,East Asia & Pacific
4,Andorra,Europe & Central Asia


In [51]:
def get_region(country):
    try:
        return regions_df.loc[regions_df['TableName'] == country]['Region'].values[0]
    except:
        return 'Other'

In [11]:
get_region("Cote d'Ivoire")

'Other'

Preview of table <strong>Indicators</strong>

In [12]:
pd.read_sql_query("SELECT * FROM Indicators LIMIT 1", connection).set_index('CountryCode').sort_index().drop('index', 1)

,CountryName,IndicatorName,IndicatorCode,Year,Value
CountryCode,,,,,
ARB,Arab World,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,133.560907


In [13]:
indicators_df = pd.read_sql_query("SELECT DISTINCT IndicatorName, IndicatorCode FROM Indicators", connection)

In [14]:
indicators_df = indicators_df.sort_values('IndicatorName').reset_index().drop('index', 1).sort_index()

Let's print them all out.

In [15]:
for index, row in indicators_df.iterrows():
    print(row['IndicatorName'] + '  ' + row['IndicatorCode'])

2005 PPP conversion factor, GDP (LCU per international $)  PA.NUS.PPP.05
2005 PPP conversion factor, private consumption (LCU per international $)  PA.NUS.PRVT.PP.05
ARI treatment (% of children under 5 taken to a health provider)  SH.STA.ARIC.ZS
Access to electricity (% of population)  EG.ELC.ACCS.ZS
Access to electricity, rural (% of rural population)  EG.ELC.ACCS.RU.ZS
Access to electricity, urban (% of urban population)  EG.ELC.ACCS.UR.ZS
Access to non-solid fuel (% of population)  EG.NSF.ACCS.ZS
Access to non-solid fuel, rural (% of rural population)  EG.NSF.ACCS.RU.ZS
Access to non-solid fuel, urban (% of urban population)  EG.NSF.ACCS.UR.ZS
Adequacy of social insurance programs (% of total welfare of beneficiary households)  per_si_allsi.adq_pop_tot
Adequacy of social protection and labor programs (% of total welfare of beneficiary households)  per_allsp.adq_pop_tot
Adequacy of social safety net programs (% of total welfare of beneficiary households)  per_sa_allsa.adq_pop_tot
Ad

1343 indicators! Which ones shall I choose?

In [153]:
interesting_indicators = {
#     'exposure_percentage': {
#         'code': 'EN.ATM.PM25.MC.ZS',
#         'name': 'population exposed to levels exceeding WHO guideline value (% of total)'},
#     'exposure_mean': {
#         'code': 'EN.ATM.PM25.MC.M3',
#         'name': 'mean annual exposure (micrograms per cubic meter)' },
    'energy_use': {
        'code': 'EG.USE.PCAP.KG.OE', 
        'name': 'Energy use (kg of oil equivalent per capita)'},
    'emission_co2': {
        'code': 'EN.ATM.CO2E.PC',  
        'name': 'CO2 emissions (metric tons per capita)'},
#     'emission_co2_tot': {
#         'code': 'EN.ATM.CO2E.KT',  
#         'name': 'CO2 emissions (kt)'},
    'population': {
        'code': 'SP.POP.TOTL',
        'name': 'population total'},
#     'gdp_percap': {
#         'code': 'NY.GDP.PCAP.CD',
#         'name': 'GDP per capita (current US$)'}
}

Let's make a data frame with only the interesting indicators instead of 1343 of them.

In [154]:
query_string = ' OR '.join(
    ["IndicatorCode='" + interesting_indicators[indicator]['code'] + "'" for indicator in interesting_indicators.keys()]
)

In [155]:
query_string

"IndicatorCode='EG.USE.PCAP.KG.OE' OR IndicatorCode='SP.POP.TOTL' OR IndicatorCode='EN.ATM.CO2E.PC'"

In [156]:
data = pd.read_sql_query("SELECT * FROM Indicators " +
                         "WHERE " + query_string, 
                          connection).set_index('CountryCode').sort_index() 

### Extract Uniform Data

Define a function to extract required data for a given country and indicator code.

In [207]:
records_wanted = [1971 + i for i in range(40)]

In [208]:
import numpy as np

In [200]:
def get_data(code, country):
    country_data = data.loc[data['CountryName'] == country]
    df = country_data.loc[country_data['IndicatorCode'] == interesting_indicators[code]['code']]
    df = df.rename(columns = {'Value': code}).set_index('Year').sort_index()
    return df[code]

In [213]:
def get_data_for_country(country):
    return pd.concat(
        [get_data(indicator, country).to_frame() for indicator in interesting_indicators], axis=1, join='inner'
    )

In [214]:
all_countries = sorted(data.CountryName.unique())
len(all_countries)

247

In [227]:
def check_is_subset(small, big):
    boolean = True
    for a in small:
        if a not in big:
            boolean = False
    return boolean

In [228]:
countries_with_good_record = []
for country in all_countries:
    records_got = get_data_for_country(country).index.values
    if (len(records_got) > len(records_wanted) and check_is_subset(records_wanted, records_got)):
        countries_with_good_record.append(country)
        
len(countries_with_good_record)

127

In [229]:
countries_with_good_record

['Albania',
 'Algeria',
 'Angola',
 'Arab World',
 'Argentina',
 'Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Benin',
 'Bolivia',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Cameroon',
 'Canada',
 'Central Europe and the Baltics',
 'Chile',
 'China',
 'Colombia',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Cuba',
 'Cyprus',
 'Denmark',
 'Dominican Republic',
 'East Asia & Pacific (all income levels)',
 'East Asia & Pacific (developing only)',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Ethiopia',
 'Euro area',
 'European Union',
 'Finland',
 'France',
 'Gabon',
 'Ghana',
 'Greece',
 'Guatemala',
 'Haiti',
 'High income',
 'High income: OECD',
 'Honduras',
 'Hong Kong SAR, China',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran, Islamic Rep.',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kenya',
 'Korea, Rep.',
 'Latin America & Caribbean (all income levels)',
 'Latin America & Caribbean (developing only

### Let's Make a Scatter Plot

In [281]:
import plotly.plotly as py
from random import randint
import colorlover as cl
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
init_notebook_mode(connected=True)

In [282]:
def get_colors(N):
    return cl.scales[str(N)]['div']['RdYlBu'];

In [283]:
def get_random_color():
    r1 = str(randint(0,255))
    r2 = str(randint(0,255))
    r3 = str(randint(0,255))
    return 'rgba(' + r1 + ', ' + r2 + ', ' + r3 + ', .9)'
get_random_color()

'rgba(148, 12, 183, .9)'

https://plot.ly/python/reference/#scatter

In [284]:
x_axis = 'energy_use'
y_axis = 'emission_co2'

In [285]:
figure = {
    'data': [], # Array of traces (dictionnaries)
    'frames': [], # Array of frames (dictionnaries)
    'layout': {}, # Dictionnary of layout properties
}

In [286]:
figure['layout'] = {
    'xaxis': {
        'type': 'log',
        'title': interesting_indicators[x_axis]['name']
    },
    'yaxis': {
        'type': 'log',
        'zeroline' : False,
        'title': interesting_indicators[y_axis]['name']

    },
    'hovermode': 'closest',
    'height': 700
}

In [287]:
def make_trace(country):
    trace_data = get_data_for_country(country).ix[records_wanted]
    return {
        'x': trace_data[x_axis].values, 
        'y': trace_data[y_axis].values,
        'name': country,
        'mode': 'lines', 
        'text': ['Year '+ str(trace_data[x_axis].index.values[i]) for i in range(len(trace_data[x_axis].index.values))],
        # Color each line with a random color.
        'line': {'width':2, 'color': get_random_color()},
        # Starting point: show only Canada.
        'visible': 'legendonly' if country != 'Canada' else True
    }

In [288]:
figure['data'] = [
    make_trace(country) for country in countries_with_good_record
]

In [289]:
py.iplot(figure)

In [290]:
figure['layout'] = {
    'xaxis': {
        'type': 'log',
        'title': interesting_indicators[x_axis]['name']
    },
    'yaxis': {
        'type': 'log',
        'zeroline' : False,
        'title': interesting_indicators[y_axis]['name']

    },
    'hovermode': 'closest',
    'height': 700
}

In [291]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

In [292]:
years = get_data_for_country(country).ix[records_wanted][x_axis].index.values

figure['layout']['slider'] = {
    'args': [
        'slider.value', {
            'duration': 400,
            'ease': 'cubic-in-out'
        }
    ],
    'initialValue': years[0],
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

# Define my_slide to be inserted into the figure
my_slider = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
my_slider['steps'] = [{
    'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'
} for year in years]

figure['layout']['sliders'] = [my_slider]


In [293]:
def get_data_by_axis(axis, countries):
    if axis == 'x':
        return [get_data_for_country(country).ix[records_wanted][x_axis] for country in countries]
    if axis == 'y':
        return [get_data_for_country(country).ix[records_wanted][y_axis] for country in countries]
    if axis == 'size':
        return [get_data_for_country(country).ix[records_wanted]['population'] for country in countries]

In [294]:
def make_frame(region, index, color):
    good_region_countries = []    
    for country in countries_with_good_record:
        if get_region(country) == region:
            good_region_countries.append(country)
    trace_data_x = [trace.values[index] for trace in get_data_by_axis('x', good_region_countries)]
    trace_data_y = [trace.values[index] for trace in get_data_by_axis('y', good_region_countries)]
    trace_data_size = [trace.values[index] for trace in get_data_by_axis('size', good_region_countries)]
    return {
        'x': trace_data_x, 
        'y': trace_data_y,
        'name': region,
        'mode': 'markers', 
        'text': [str(country) for country in good_region_countries],
        # Color each line with a random color.
        'marker': {
            'color': color,
            'sizemode': 'area',
            'sizeref': 500000,
            'size': trace_data_size
        },
        # Starting point: show only Canada.
        'visible': False if 'income' in country else True
    }

In [295]:
unique_regions = sorted(regions_df['Region'].dropna().unique())
unique_regions

['East Asia & Pacific',
 'Europe & Central Asia',
 'Latin America & Caribbean',
 'Middle East & North Africa',
 'North America',
 'South Asia',
 'Sub-Saharan Africa']

In [296]:
colors = {}
color_array = get_colors(len(unique_regions))
for region, color in zip(unique_regions, color_array):
    colors[region] = color

In [297]:
figure['data'] = [
    make_frame(region, 0, colors[region]) for region in unique_regions 
]

In [298]:
figure['frames'] = [
    {
        'data': [
            make_frame(region, index, colors[region]) for region in unique_regions
        ],
        'name': str(years[index])
    } for index in range(len(years))
]

In [299]:
iplot(figure)